In [1]:
%matplotlib inline
import os
import warnings

import critical_loads as cl
import gdal
import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid

# warnings.simplefilter("ignore")
plt.style.use("ggplot")

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(database="critical_loads")

Connection successful.


# Critical loads for water: sensitivity analysis

This notebook explores the sensitivity of the exceedance calculations (SSWC and FAB) to changes in the input critical loads parameters. See e-mail from Kari received 17.11.2020 at 10.01 for details.

## 1. User input

Choose the deposition dataset and resolution you wish to work with (see notebook 01 for a list of available `series_ids`) and set the parameter combinations to explore. For everything here, we will use "standard" `BC0` and deposition data from 2012 to 2016 (new grid), which is series ID 28.

In [3]:
ser_id = 28
short_name = "1216"
cell_size = 120
bc0 = "BC0"
base_fold = f"/home/jovyan/shared/critical_loads/raster/sensitivity_analysis"

# Values to try for each parameter
param_dict = {
    "Runoff": [1, 1.03, 1.05],
    "Ni": [3.57, 7.14, 14.23],
    "NO3N": [1, 0.5, 0.1],
    "Fde": [0.1, 0.2, 0.5],
}

# Whether values in 'param_dict' are absolute parameter values ('abs') or multiplication factors
# to apply to existing parameters ('fac')
param_type = {
    "Runoff": "fac",
    "Ni": "abs",
    "NO3N": "fac",
    "Fde": "abs",
}

## 2. Loop over parameter combinations

The code below loops over all the parameter combinations defined above. It's messy/hacky as it's mostly copy-pasted from other notebooks, but should work OK.

If we save all the critical loads grids, **each run generates around 240 MB of data and takes about 1 minute 20 seconds. Care is therefore needed if performaing a large number of runs**.

In [4]:
# Generate param combos
param_grid = ParameterGrid(param_dict)

print(f"WARNING: You have chosen {len(param_grid)} parameter combinations.")
print(f"         The code below will generate ~{len(param_grid)*0.240:.2f} GB of data "
      f"and will take around {len(param_grid)*1.33/60:.2f} hours to complete.")

         The code below will generate ~19.44 GB of data and will take around 1.80 hours to complete.


In [5]:
# Loop over combos
df_list = []
for idx, params in enumerate(param_grid):
    run_id = idx + 1
    print(
        "########################################################################################"
    )
    print(f"Processing run {run_id:03d}\n")
    # Read required data from db
    par_df = pd.read_sql(
        "SELECT id as parameter_id, name, class FROM water.parameter_definitions",
        eng,
    )
    df = pd.read_sql("SELECT * FROM water.blr_required_parameters", eng)

    # Restructure
    df = pd.merge(df, par_df, how="left", on="parameter_id")
    del df["parameter_id"], df["class"]
    df = df.pivot(index="region_id", columns="name", values="value")
    df.index.name = "Region_id"
    df.reset_index(inplace=True)
    df.columns.name = ""

    # Add optional pars with default values
    default_vals = {
        "Catch_area": 1,
        "Lake_area": 0.05,
        "Forest_area": 0.95,
        "Ni": 3.57,
        "Fde": 0.1,
        "SN": 5,
        "SS": 0.5,
    }
    for key in default_vals.keys():
        df[key] = default_vals[key]

    # Update params
    for key in params.keys():
        if param_type[key] == "fac":
            # Apply change factor
            df[key] = df[key] * params[key]
        elif param_type[key] == "abs":
            # Set value explicitly
            df[key] = params[key]
        else:
            raise ValueError("Parameter type not recognised (must be 'fac' or 'abs').")

    # Split into required and optional params
    req_df = df[
        [
            "Region_id",
            "Ca",
            "Cl",
            "K",
            "Mg",
            "NO3N",
            "Na",
            "Nupt",
            "Runoff",
            "SO4",
            "TOC",
        ]
    ].copy()
    opt_df = df[
        ["Region_id", "Catch_area", "Lake_area", "Forest_area", "Ni", "Fde", "SN", "SS"]
    ].copy()

    # Rasterise critical loads for water
    cl_fold = os.path.join(base_fold, f"run_{run_id:03d}/water")
    if not os.path.exists(cl_fold):
        os.makedirs(cl_fold)
    cl.rasterise_water_critical_loads(
        eng,
        cl_fold,
        cell_size,
        bc0=bc0,
        req_df=req_df,
        opt_df=opt_df,
        mag_df=None,
        df_to_csv=True,
    )

    # SSWC
    ex_fold = os.path.join(base_fold, f"run_{run_id:03d}/exceedance")
    if not os.path.exists(ex_fold):
        os.makedirs(ex_fold)
    sswc_df = cl.calculate_water_exceedance_sswc(
        ser_id,
        short_name,
        cl_fold,
        ex_fold,
        cell_size=cell_size,
        bc0=bc0,
        neg_to_zero=True,
    )

    sswc_path = os.path.join(
        ex_fold, f"sswc_ex_meqpm2pyr_{short_name}_{cell_size}m.tif"
    )
    sswc_ex, ndv, epsg, extent = nivapy.spatial.read_raster(sswc_path)
    sswc_ex = sswc_ex.astype(np.float32)
    sswc_ex[sswc_ex == ndv] = np.nan

    # FAB
    # Read CL arrays
    array_dict = {}
    for par in [
        "clminn_meqpm2pyr",
        "clmins_meqpm2pyr",
        "clmaxnoaa_meqpm2pyr",
        "clmaxsoaa_meqpm2pyr",
    ]:
        # Read tif
        tif_path = os.path.join(cl_fold, f"{par}_{cell_size}m.tif")
        data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)

        # Set NDV
        data[data == ndv] = np.nan

        # Add to dict
        array_dict[par] = data

    # Read dep arrays
    dep_path = r"/home/jovyan/shared/critical_loads/raster/deposition"
    for par in ["ndep_mgpm2pyr", "sdep_mgpm2pyr"]:
        # Read tif
        tif_path = os.path.join(dep_path, f"{par}_{short_name}_{cell_size}m.tif")
        data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
        data = data.astype(np.float32)

        # Set NDV
        data[data == ndv] = np.nan

        # Add to dict
        array_dict[par] = data

    # Extract arrays from dict
    cln_min = array_dict["clminn_meqpm2pyr"]
    cln_max = array_dict["clmaxnoaa_meqpm2pyr"]
    cls_min = array_dict["clmins_meqpm2pyr"]
    cls_max = array_dict["clmaxsoaa_meqpm2pyr"]
    dep_n = array_dict["ndep_mgpm2pyr"] / 14  # Convert to meq
    dep_s = array_dict["sdep_mgpm2pyr"] * 2 / 32.06  # Convert to meq

    # Estimate exceedances
    ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
        cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
    )

    # Get exceeded area
    ex = ex_n + ex_s
    fab_ex_area = np.count_nonzero(ex > 0) * cell_size * cell_size / 1.0e6
    nor_area = np.count_nonzero(~np.isnan(dep_s)) * cell_size * cell_size / 1.0e6
    fab_ex_pct = 100 * fab_ex_area / nor_area

    # Build df and tidy
    fab_ex_df = pd.DataFrame(
        {
            "exceeded_area_km2": fab_ex_area,
            "total_area_km2": nor_area,
            "exceeded_area_pct": fab_ex_pct,
        },
        index=[0],
    )

    fab_ex_df = fab_ex_df.round(0).astype(int)
    fab_ex_df["series_id"] = ser_id
    fab_ex_df["medium"] = "water_fab"

    fab_ex_df = fab_ex_df[
        [
            "series_id",
            "medium",
            "total_area_km2",
            "exceeded_area_km2",
            "exceeded_area_pct",
        ]
    ]

    # Save rasters
    snap_tif = (
        f"/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif"
    )

#     n_tif = os.path.join(ex_fold, f"fab_exn_meqpm2pyr_{short_name}_{cell_size}m.tif")
#     cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

#     s_tif = os.path.join(ex_fold, f"fab_exs_meqpm2pyr_{short_name}_{cell_size}m.tif")
#     cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

    ns_tif = os.path.join(ex_fold, f"fab_exns_meqpm2pyr_{short_name}_{cell_size}m.tif")
    cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

#     reg_tif = os.path.join(ex_fold, f"fab_ex_reg_id_{short_name}_{cell_size}m.tif")
#     cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)

    # Assign run ID to params
    params["run_id"] = run_id

    params["norway_area_km2"] = nor_area
    
    # Add exceedance stats
    sswc_ex_area = sswc_df.loc[0, "exceeded_area_km2"]
    params["sswc_ex_area_km2"] = sswc_ex_area

    sswc_ex_pct = sswc_df.loc[0, "exceeded_area_pct"]
    params["sswc_ex_area_pct"] = sswc_ex_pct

    assert np.nanmin(sswc_ex) == 0
    params["sum_sswc_ex_Meqpyr"] = np.nansum(sswc_ex) * 120 * 120 / 1e9

    params["fab_ex_area_km2"] = fab_ex_area

    params["fab_ex_area_pct"] = fab_ex_pct

    assert np.nanmin(ex) == 0
    params["sum_fab_ex_Meqpyr"] = np.nansum(ex) * 120 * 120 / 1e9

    # Build summary output
    df_list.append(pd.DataFrame(params, index=[0]))


# Build dataframe of run details
res_df = pd.concat(df_list, axis="rows").reset_index(drop=True)
par_cols = list(res_df.columns)
par_cols.remove("run_id")
res_df = res_df[["run_id"] + par_cols]

csv_path = os.path.join(base_fold, "sensitivity_analysis_summary.csv")
res_df.to_csv(csv_path, index=False)
res_df

########################################################################################
Processing run 001

Rasterising claoaa_meqpm2pyr...
Rasterising eno3_flux_meqpm2pyr...
Rasterising clminn_meqpm2pyr...
Rasterising clmaxnoaa_meqpm2pyr...
Rasterising clmaxsoaa_meqpm2pyr...
Rasterising anclimit_ueqpl...
Rasterising anclimitoaa_ueqpl...
Rasterising bc0_ueqpl...
Rasterising clmins_meqpm2pyr...
Rasters saved to:
    /home/jovyan/shared/critical_loads/raster/sensitivity_analysis/run_001/water
Exceedance grid saved to:
    /home/jovyan/shared/critical_loads/raster/sensitivity_analysis/run_001/exceedance/sswc_ex_meqpm2pyr_1216_120m.tif
########################################################################################
Processing run 002

Rasterising claoaa_meqpm2pyr...
Rasterising eno3_flux_meqpm2pyr...
Rasterising clminn_meqpm2pyr...
Rasterising clmaxnoaa_meqpm2pyr...
Rasterising clmaxsoaa_meqpm2pyr...
Rasterising anclimit_ueqpl...
Rasterising anclimitoaa_ueqpl...
Rasterising bc0_ue

,run_id,Fde,NO3N,Ni,Runoff,norway_area_km2,sswc_ex_area_km2,sswc_ex_area_pct,sum_sswc_ex_Meqpyr,fab_ex_area_km2,fab_ex_area_pct,sum_fab_ex_Meqpyr
0,1,0.1,1.0,3.57,1.00,322183.8576,23225,7,290.325773,61055.7264,18.950585,1558.989937
1,2,0.1,1.0,3.57,1.03,322183.8576,22907,7,292.187434,59669.6832,18.520383,1523.860769
2,3,0.1,1.0,3.57,1.05,322183.8576,22410,7,293.615338,58761.6624,18.238550,1501.310713
3,4,0.1,1.0,7.14,1.00,322183.8576,23225,7,290.325773,55305.5184,17.165825,1391.589616
4,5,0.1,1.0,7.14,1.03,322183.8576,22907,7,292.187434,54307.7424,16.856134,1359.918017
...,...,...,...,...,...,...,...,...,...,...,...,...
76,77,0.5,0.1,7.14,1.03,322183.8576,8660,3,37.749438,37448.7120,11.623398,678.290397
77,78,0.5,0.1,7.14,1.05,322183.8576,8485,3,36.494226,36768.9744,11.412420,663.895665
78,79,0.5,0.1,14.23,1.00,322183.8576,9120,3,39.772753,33370.1712,10.357493,560.146088
79,80,0.5,0.1,14.23,1.03,322183.8576,8660,3,37.749438,32036.9328,9.943680,541.549095
